Here a complete example of fitting is shown.  
This is intended only for explainatory purpose, then fine tuning or correctness are not the aims of this notebook. You can alway make it better submitting a PR 😉

Workspace setup

In [ ]:
import os
from pathlib import Path
import logging
logging.basicConfig()
log = logging.getLogger()
log.setLevel(logging.INFO)
pil_logger = logging.getLogger('PIL')
pil_logger.setLevel(logging.INFO)

In [ ]:
root = Path('./../') # quite the same as str

# Set an example dir for images files
MOCKS = root.joinpath('./../cytokinin/cytokinin/tests/mocks/')
IMGS = MOCKS/'imgs' # this is another Path object
os.listdir(str(IMGS))

Now let's rock

In [ ]:
import cytokinin as ck
from cytokinin.data import take_data

In [ ]:
# Create a Data from two roots: dogs and stones
dogs = take_data('images').store_filesnames_from_folder(IMGS.joinpath('dog'))
stones = take_data('images').store_filesnames_from_folder(IMGS.joinpath('stone'))
dands = dogs.copy().add_from_data(stones)

csv_url = MOCKS/'labels'/'dogsandstones_labes.csv'
dands.label_from_csv(csv_url, col='Y')
print(dands)

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

Just define a Pytorch [tranforms object](https://pytorch.org/docs/stable/torchvision/transforms.html) for the Data object and you'll be ready to pass it to a Pytorch [Dataloader](https://pytorch.org/docs/stable/data.html).

In [ ]:
transformations = transforms.Compose([
    transforms.Resize((224,224),interpolation=2), # NEEDED if you want to use *batch_size*
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dands.set_transforms(transformations)
print(dands.torch_tsfm)

In [ ]:
bs = 8
train_loader = DataLoader(dands, batch_size=bs, shuffle=True)
train_loader

In [ ]:
from torch import LongTensor
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
log.setLevel(logging.INFO)

model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,bias=False)
model.avgpool = nn.AdaptiveAvgPool2d(1)
model.fc = nn.Linear(512 * models.resnet.BasicBlock.expansion, 2)
net = model
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

nepochs = 2
for epoch in range(nepochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        prp = i*100/len(train_loader)
        print(f'{prp}% of epoch {epoch}/{nepochs}')
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        labels = labels.type(LongTensor)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training done!')